In [1]:
from collections import defaultdict
import os, os.path
import re
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk.resolve_layer_dag import make_resolver
from estnltk.taggers import VabamorfTagger
from estnltk.taggers import VabamorfAnalyzer
from estnltk.taggers import VabamorfCorpusTagger, CorpusBasedMorphDisambiguator
from estnltk import Text

In [2]:
#korpusepõhine ühestamine
#NB! Tekstid tuleb enne tagimist grupeerida
korpusepohine = VabamorfCorpusTagger( output_layer='morph_korpusepohine' )

In [3]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EDT_hastaalfinaal'
assert os.path.isdir( input_dir )
loaded_texts_edt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_edt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_edt-ud-dev_000_am.json
 Loaded  et_edt-ud-dev_001_am.json
 Loaded  et_edt-ud-dev_002_am.json
 Loaded  et_edt-ud-dev_003_am.json
 Loaded  et_edt-ud-dev_004_am.json
 Loaded  et_edt-ud-dev_006_am.json
 Loaded  et_edt-ud-dev_007_am.json
 Loaded  et_edt-ud-dev_008_am.json
 Loaded  et_edt-ud-test_009_am.json
 Loaded  et_edt-ud-test_010_am.json
 Loaded  et_edt-ud-test_011_am.json
 Loaded  et_edt-ud-test_013_am.json
 Loaded  et_edt-ud-test_014_am.json
 Loaded  et_edt-ud-train_015_am.json
 Loaded  et_edt-ud-train_016_am.json
 Loaded  et_edt-ud-train_017_am.json
 Loaded  et_edt-ud-train_018_am.json
 Loaded  et_edt-ud-train_019_am.json
 Loaded  et_edt-ud-train_020_am.json
 Loaded  et_edt-ud-train_021_am.json
 Loaded  et_edt-ud-train_022_am.json
 Loaded  et_edt-ud-train_023_am.json
 Loaded  et_edt-ud-train_024_am.json
 Loaded  et_edt-ud-train_025_am.json
 Loaded  et_edt-ud-train_027_am.json
 Loaded  et_edt-ud-train_028_am.json
 Loaded  et_edt-ud-train_029_am.json
 Loaded  et_edt-ud-tra

In [4]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EWT_hastaalfinaal'
assert os.path.isdir( input_dir )
loaded_texts_ewt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_ewt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_ewt-ud-dev_000_am.json
 Loaded  et_ewt-ud-dev_001_am.json
 Loaded  et_ewt-ud-dev_002_am.json
 Loaded  et_ewt-ud-dev_003_am.json
 Loaded  et_ewt-ud-dev_004_am.json
 Loaded  et_ewt-ud-test_005_am.json
 Loaded  et_ewt-ud-test_006_am.json
 Loaded  et_ewt-ud-test_007_am.json
 Loaded  et_ewt-ud-test_008_am.json
 Loaded  et_ewt-ud-test_009_am.json
 Loaded  et_ewt-ud-test_010_am.json
 Loaded  et_ewt-ud-test_011_am.json
 Loaded  et_ewt-ud-train_012_am.json
 Loaded  et_ewt-ud-train_013_am.json
 Loaded  et_ewt-ud-train_014_am.json
 Loaded  et_ewt-ud-train_015_am.json
 Loaded  et_ewt-ud-train_016_am.json
 Loaded  et_ewt-ud-train_017_am.json
 Loaded  et_ewt-ud-train_018_am.json
 Loaded  et_ewt-ud-train_019_am.json
 Loaded  et_ewt-ud-train_020_am.json
 Loaded  et_ewt-ud-train_021_am.json
 Loaded  et_ewt-ud-train_022_am.json
 Loaded  et_ewt-ud-train_023_am.json
 Loaded  et_ewt-ud-train_024_am.json
 Loaded  et_ewt-ud-train_025_am.json
 Loaded  et_ewt-ud-train_026_am.json
 Loaded  et_ewt-ud

In [12]:
#jaotab korpuse žanripõhiselt eraldi listidesse
aja = [] #ajakirjandus
tea = [] #teadus
ilu = [] #ilukirjandus
for t in loaded_texts_edt_am:
    file_id = t.meta['file_prefix']
    if re.match('^tea', file_id):
        tea.append(t)
    elif re.match('^aja', file_id):
        aja.append(t)
    elif re.match('^ilu', file_id):
        ilu.append(t)
        
foorum = [] 
ewtb = [] #estonian web treebank (vanad tekstid)
kom = [] #kommentaarium
for t in loaded_texts_ewt_am:
    file_id = t.meta['file_prefix']
    if 'foorum' in file_id:
        foorum.append(t)
    elif 'ewtb' in file_id:
        ewtb.append(t)
    elif 'kom' in file_id:
        kom.append(t)

In [13]:
#KORPUSEPÕHISED
korpusepohine.tag(aja)
korpusepohine.tag(tea)
korpusepohine.tag(ilu)
korpusepohine.tag(foorum)
korpusepohine.tag(ewtb)
korpusepohine.tag(kom)

edt_korpusepohistega = aja + ilu + tea
ewt_korpusepohistega = foorum + ewtb + kom

In [20]:
output_dir = 'Automaatmargendusega_EDT_hastaalfinaal2'
if not os.path.isdir( output_dir ):
    os.makedirs(output_dir)
assert os.path.isdir( output_dir )

In [21]:
# Load gold standard texts and add pre-annotations
i = -1
for tekst in edt_korpusepohistega:
    i += 1
    if fname.endswith('.json'):
        fname = tekst.meta['file']
        fname2 = '_' + str(i) + '_am.json' 
        fname = fname.replace('.conllu', fname2)
        fpath = os.path.join(output_dir, fname)
        text_to_json(tekst, file=fpath)
        print(' Loaded, pre-annotated and written', fname)

 Loaded, pre-annotated and written et_edt-ud-dev_0_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_1_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_2_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_3_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_4_am.json
 Loaded, pre-annotated and written et_edt-ud-test_5_am.json
 Loaded, pre-annotated and written et_edt-ud-test_6_am.json
 Loaded, pre-annotated and written et_edt-ud-test_7_am.json
 Loaded, pre-annotated and written et_edt-ud-train_8_am.json
 Loaded, pre-annotated and written et_edt-ud-train_9_am.json
 Loaded, pre-annotated and written et_edt-ud-train_10_am.json
 Loaded, pre-annotated and written et_edt-ud-train_11_am.json
 Loaded, pre-annotated and written et_edt-ud-train_12_am.json
 Loaded, pre-annotated and written et_edt-ud-train_13_am.json
 Loaded, pre-annotated and written et_edt-ud-train_14_am.json
 Loaded, pre-annotated and written et_edt-ud-train_15_am.json
 Loaded, pre-annotated and writ

In [22]:
output_dir = 'Automaatmargendusega_EWT_hastaalfinaal2'
if not os.path.isdir( output_dir ):
    os.makedirs(output_dir)
assert os.path.isdir( output_dir )

In [23]:
# Load gold standard texts and add pre-annotations
i = -1
for tekst in ewt_korpusepohistega:
    i += 1
    if fname.endswith('.json'):
        fname = tekst.meta['file']
        fname2 = '_' + str(i) + '_am.json' 
        fname = fname.replace('.conllu', fname2)
        fpath = os.path.join(output_dir, fname)
        text_to_json(tekst, file=fpath)
        print(' Loaded, pre-annotated and written', fname)

 Loaded, pre-annotated and written et_ewt-ud-dev_0_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_1_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_2_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_3_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_4_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_5_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_6_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_7_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_8_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_9_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_10_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_11_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_12_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_13_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_14_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_15_am.json
 Loaded, pre-annotated and written 